<a href="https://colab.research.google.com/github/aimyapp/mapleEventNotice/blob/main/mapleEventNotice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title System Setup
import subprocess

# 必要なパッケージをインストール
subprocess.run("sudo apt-get install libvulkan1", shell=True, check=True)

# その他のコマンドを続けて実行
subprocess.run("sudo apt-get -f install", shell=True, check=True)
subprocess.run("sudo apt -y update", shell=True, check=True)
subprocess.run("sudo pip install requests selenium", shell=True, check=True)

# ダウンロード用パッケージのインストール
subprocess.run("sudo apt install -y wget curl unzip", shell=True, check=True)

# 依存関係のインストール
subprocess.run("sudo wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb", shell=True, check=True)
subprocess.run("sudo dpkg -i libu2f-udev_1.1.4-1_all.deb", shell=True, check=True)

# Chromeのインストール
subprocess.run("sudo wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb", shell=True, check=True)
subprocess.run("sudo dpkg -i google-chrome-stable_current_amd64.deb", shell=True, check=True)

# ChromeDriverのインストール
subprocess.run("sudo wget -N https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/linux64/chromedriver-linux64.zip -P /tmp/", shell=True, check=True)
subprocess.run("sudo unzip -o /tmp/chromedriver-linux64.zip -d /tmp/", shell=True, check=True)
subprocess.run("sudo chmod +x /tmp/chromedriver-linux64/chromedriver", shell=True, check=True)
subprocess.run("sudo mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver", shell=True, check=True)


In [ ]:
#@title Main Feature
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep

# 環境を判別
try:
    import google.colab
    is_colab = True
except ImportError:
    is_colab = False

is_github_actions = os.getenv("GITHUB_ACTIONS") == "true"

# アクセスキーの取得方法を分岐
if is_colab:
    print("Running on Google Colab.")
    from google.colab import userdata
    access_token = userdata.get('ACCESS_TOKEN')
   # user_id = userdata.get('USER_ID')
    group_id = userdata.get('GROUP_ID')
elif is_github_actions:
    print("Running on GitHub Actions.")
    access_token = os.getenv('ACCESS_TOKEN')
   # user_id = os.getenv('USER_ID')
    group_id = userdata.get('GROUP_ID')
else:
    raise EnvironmentError("This script is designed to run on Google Colab or GitHub Actions only.")

# Chromeオプションの設定
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--remote-debugging-port=9222')

# ChromeDriverのパスを設定
chrome_service = Service('/usr/local/bin/chromedriver')

# ブラウザを起動
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# 指定されたURLを開く
driver.get('https://maplestory.nexon.co.jp/notice/list/event/')

# class="card-list"の要素を取得
card_list = driver.find_element(By.CLASS_NAME, 'card-list')

# ul配下のすべてのli要素を取得
list_items = card_list.find_elements(By.TAG_NAME, 'li')

# 各li要素の情報を収集
messages = []
for item in list_items:
    link = item.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    title = item.find_element(By.CLASS_NAME, 'title').text
    date = item.find_element(By.CLASS_NAME, 'date').text
    print(f"Title: {title}\nDate: {date}\nLink: {href}\n")
    messages.append(f"{title}\n{date}\n{href}\n")

# LINE Messaging APIを使って通知を送信

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


# お知らせ開始の挨拶
data = {
    # 個人LINEに通知を送る場合はuser_idを指定
    # "to": user_id,
    # グループLINEに通知を送る場合はuser_idを指定
    "to": group_id,
    "messages": [
        {
          "type":"text",
          "text":"開催中のイベントぷるー!!!!"
        }
    ]
}
response = requests.post("https://api.line.me/v2/bot/message/push", headers=headers, json=data)

# 開催中のイベントを5件ごとに一つにまとめて送信
for i in range(0, len(messages), 5):
    batch = messages[i:i+5]
    combined_message = "\n".join(batch)
    notice_data = {
        "to": group_id,
        "messages": [
            {
                "type": "text",
                "text": combined_message
            }
        ]
    }
    response = requests.post("https://api.line.me/v2/bot/message/push", headers=headers, json=notice_data)
    # ステータスコードの確認
    print(f"Status Code: {response.status_code}")

    # エラーメッセージの確認、デバッグ用
    #try:
    #    response_data = response.json()
    #   print(f"Response JSON: {response_data}")
    #except ValueError:
    #    print("Response content is not in JSON format")

    # レスポンスの内容の確認
    #print(f"Response Text: {response.text}")

# ブラウザを終了
driver.quit()
